### The temporal network class

In this notebook we explore how to load the coherence connectivity data with the temporal network class

In [1]:
# Path to the toolbox
import sys; sys.path.insert(1, '/home/vinicius/storage1/projects/GrayData-Analysis')
import os
import xarray as xr
import pandas as pd
import numpy  as np
# GDa
from   GDa.temporal_network import temporal_network
import matplotlib.pyplot as plt 

### Instantiate temporal network

In [2]:
net = temporal_network(coh_file='coh_k_0.3_morlet.nc', coh_sig_file=None, 
                 date='141017', trial_type=[1], behavioral_response=[1])

In [3]:
net.convert_to_adjacency()

In [ ]:
out = net.super_tensor.groupby("roi").mean("roi").interp({"freqs":np.linspace(3,75,20)}, "quadratic")

In [ ]:
out.mean(dim=("times","trials")).plot(x="freqs", hue="roi", color="b", lw=1);
plt.legend()